In [1]:
"""overall_inflation_predictor.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1MTH7HkjV8DEhIAU2JZXE-sRIaoDJM21J
"""

'overall_inflation_predictor.ipynb\n\nAutomatically generated by Colab.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/1MTH7HkjV8DEhIAU2JZXE-sRIaoDJM21J\n'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl

In [3]:
df = pd.read_csv('world_bank_data_2025.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'world_bank_data_2025.csv'

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.columns.tolist()

In [ ]:
df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()
print(df.columns)

In [ ]:
df.rename(columns={
    'country_name': 'country',
    'country_id': 'country_code',
    'year': 'year',
    'inflation_(cpi_%)': 'cpi',
    'gdp_(current_usd)': 'gdp',
    'gdp_per_capita_(current_usd)': 'gdp_per_capita',
    'unemployment_rate_(%)': 'unemployment_rate',
    'interest_rate_(real,_%)': 'real_interest_rate',
    'inflation_(gdp_deflator,_%)': 'gdp_deflator',
    'gdp_growth_(%_annual)': 'gdp_growth',
    'current_account_balance_(%_gdp)': 'current_account_balance',
    'government_expense_(%_of_gdp)': 'gov_expense',
    'government_revenue_(%_of_gdp)': 'gov_revenue',
    'tax_revenue_(%_of_gdp)': 'tax_revenue',
    'gross_national_income_(usd)': 'gni',
    'public_debt_(%_of_gdp)': 'public_debt',
}, inplace=True)

In [ ]:
df.columns

In [ ]:
df.drop(['country_code'], axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
for col in df.select_dtypes(include=np.number).columns:
    df.fillna({col: df[col].median()}, inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
plt.figure(figsize=(10, 5))
df.groupby('year')['cpi'].mean().plot(marker='o')
plt.title('Average Global Inflation (CPI %) Over Time')
plt.ylabel('Inflation (CPI %)')
plt.xlabel('Year')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.drop(columns=['country']).corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Between Economic Indicators')
plt.show()

In [ ]:
# Top 10
top10 = df.groupby('country')['cpi'].mean().sort_values(ascending=False).head(10)
top10.plot(kind='bar', figsize=(10,5), title='Top 10 Countries with Highest Avg Inflation (CPI %)', color='crimson')
plt.ylabel('Avg Inflation')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Bottom 10
bottom10 = df.groupby('country')['cpi'].mean().sort_values().head(10)
bottom10.plot(kind='bar', figsize=(10,5), title='Bottom 10 Countries with Lowest Avg Inflation (CPI %)', color='seagreen')
plt.ylabel('Avg Inflation')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df['cpi'], bins=30, kde=True, color='steelblue')
plt.title('Distribution of Inflation (CPI %)')
plt.xlabel('Inflation (CPI %)')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
inflation_bins = [-10, 0, 2, 5, 10, 20, 50, 100, 1000]
bin_labels = ['<0', '0–2', '2–5', '5–10', '10–20', '20–50', '50–100', '100+']
df['inflation_range'] = pd.cut(df['cpi'], bins=inflation_bins, labels=bin_labels)
range_counts = df['inflation_range'].value_counts().sort_index()
plt.figure(figsize=(10, 5))
sns.barplot(x=range_counts.index, y=range_counts.values, palette='magma')
plt.title('Number of Country-Year Entries per Inflation Range')
plt.xlabel('Inflation Range (CPI %)')
plt.ylabel('Number of Entries')
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
key_features = ['gdp', 'gdp_per_capita', 'unemployment_rate', 'real_interest_rate',
                'public_debt', 'gov_expense', 'tax_revenue']

In [ ]:
for col in key_features:
    plt.figure(figsize=(6, 3))
    sns.histplot(df[col], kde=True, bins=30, color='skyblue')
    plt.title(f'Distribution of {col.replace("_", " ").title()}')
    plt.xlabel(col)
    plt.tight_layout()
    plt.show()

In [ ]:
df['debt_to_gdp_ratio'] = df['public_debt'] / df['gdp']
df['gdp_growth_per_capita'] = df['gdp_growth'] / df['gdp_per_capita']
df['gov_balance'] = df['gov_revenue'] - df['gov_expense']

In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
for col in ['debt_to_gdp_ratio', 'gdp_growth_per_capita', 'gov_balance']:
    if col in df.columns:
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)

In [ ]:
df.columns

In [ ]:
y = df['cpi']
feature_columns = [
    'gdp', 'gdp_per_capita', 'unemployment_rate', 'real_interest_rate',
    'gdp_deflator', 'gdp_growth', 'current_account_balance', 'gov_expense',
    'gov_revenue', 'tax_revenue', 'gni', 'public_debt',
    'debt_to_gdp_ratio', 'gdp_growth_per_capita', 'gov_balance'
]
X = df[feature_columns]

In [ ]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=feature_columns)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled_df, y, test_size=0.2, random_state=42
)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("Linear Regression Results:")
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_lr)))
print("R^2 Score:", r2_score(y_test, y_pred_lr))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("\nRandom Forest Results:")
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))
print("R^2 Score:", r2_score(y_test, y_pred_rf))

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.3,
    reg_lambda=0.7,
    random_state=42
)

In [ ]:
xgb.fit(X_train, y_train)

In [ ]:
y_pred_xgb = xgb.predict(X_test)

In [ ]:
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2_xgb = r2_score(y_test, y_pred_xgb)

In [ ]:
print("XGBoost Regressor Results")
print("-------------------------")
print(f"RMSE     : {rmse_xgb:.4f}")
print(f"R² Score : {r2_xgb:.4f}")

In [ ]:
!pip install optuna

In [ ]:
import optuna
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 2000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0)
    }
    model = XGBRegressor(**params, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=150, show_progress_bar=True)

In [ ]:
# Step 5: Train final model using best hyperparameters
print("Best RMSE:", study.best_value)
print("Best Parameters:", study.best_params)

In [ ]:
best_params = study.best_params
final_model = XGBRegressor(**best_params, random_state=42)
final_model.fit(X_train, y_train)

In [ ]:
# Evaluate
y_pred = final_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

In [ ]:
print("\nXGBoost Results with Optuna:")
print("RMSE:", rmse)
print("R² Score:", r2)